In [1]:
import requests
import os
import csv
import pandas as pd
import json
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from random import randint
from datetime import datetime

In [2]:
### 3901.csv in Dataframe (EGID)

egid = pd.read_csv('EGID/3901/3901.csv', delimiter=';')

# Spalte URL hinzufügen
egid['URL'] = 'https://www.housing-stat.ch/regbl/resources/public/geb_public/' + egid['EGID'].astype('str') + '?lang=de'


In [3]:
# URL herunterladen, als pdf abspeichern

# alle URLS in Liste, für download in Schlaufe
url_liste = egid.values.tolist()

urls = []
url_egid = {}

for url_list in url_liste:
    # Dictionary je Zeile aus Dataframe
    einzelne_egid = url_list[0]
    einzelne_url = url_list[13]
    
    url_egid = {'EGID': einzelne_egid, 'URL': einzelne_url}
    urls.append(url_egid)

# jede URL runterladen und als pdf abspeichern
for url in tqdm(urls):
    
    inhalt = requests.get(url['URL'], allow_redirects=True)
    time.sleep(5)
    with open('PDF/GEB_PUBLIC' + str(url['EGID']) + '.pdf', 'wb') as pdffile:
        pdffile.write(inhalt.content)
        
        

100%|██████████| 5107/5107 [8:14:40<00:00,  5.81s/it]   


In [4]:
# PDF auslesen
# EGID, Gebäudenummer, Koordinaten etc.

import os
import textract
from tqdm import tqdm
import pandas as pd


# leere Liste für alle Gebäude
alle_gebaeude = []

# PDF in Liste
pdfs = [f for f in os.listdir('PDF/') if f.endswith('.pdf') or f.endswith('.PDF')]

for pdf in tqdm(pdfs):

    #text = textract.process('PDF/' + pdf, method='pdfminer', encoding='ISO-8859-1')
    text = textract.process('PDF/' + pdf, method='pdfminer', encoding='ISO-8859-1')
    text = text.decode("ISO-8859-1").replace("\n", "xxxxxxxxxx").split("xxxxxxxxxx")


    
    #minidict grad mit den gewünschten elementen
    gebaeude = {'Grundstueck-Nummer': int(text[13].replace(' ','')),
               'EGID': int(text[44].replace(' ','')),
               'Gebaeude-Koordinate X': int(text[18].replace(' ','')),
               'Gebaeude-Koordinate Y': int(text[22].replace(' ','')),
               'Gebaeude-Baujahr': int(text[28].replace(' ','')),
               'Gebaeude-Kategorie': text[55] + " " + text[56],
                'Gebaeude-Klasse-Nummer': text[50],
               'Gebaeude-Klasse': text[57] + " " + text[58],
               'Gebaeude-Flaeche': int(text[59].replace(' ','')),
               'Gebaeude-Geschosse': int(text[64].replace(' ','')),
               'Gebaeude-Strasse': text[81],
               'Gebaeude-Hausnummer': text[84],
               'Gebaeude-PLZ': text[87],
               'Gebaeude-Ort': text[90]}
    
    alle_gebaeude.append(gebaeude)
    
 


# csv abspeichern
file = pd.DataFrame(alle_gebaeude)            
headernames = ['Grundstueck-Nummer', 'EGID', 'Gebaeude-Koordinate X', 'Gebaeude-Koordinate Y',
               'Gebaeude-Baujahr', 'Gebaeude-Kategorie', 'Gebaeude-Klasse-Nummer',
               'Gebaeude-Klasse', 'Gebaeude-Flaeche', 'Gebaeude-Geschosse', 'Gebaeude-Strasse',
               'Gebaeude-Hausnummer', 'Gebaeude-PLZ', 'Gebaeude-Ort']

file.to_csv('Gebäude_Chur.csv', header=headernames, sep=";", index=False)


  0%|          | 1/5011 [00:01<1:29:30,  1.07s/it]


ValueError: invalid literal for int() with base 10: ''

In [7]:
text[81]

'Pulvermühlestrasse'